In [99]:
import requests
import json
import os
from tenacity import retry,wait_random_exponential,stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo"
RED_CLR = "red"
GREEN_CLR = "green"
BLUE_CLR = "blue"
MAGENTA_CLR = "magenta"

SYSTEM_ROLE = "system"
USER_ROLE = "user"
ASSISTANT_ROLE = "assistant"
FUNCTION_ROLE = "function"

role_to_color = {
    SYSTEM_ROLE:RED_CLR,
    USER_ROLE:GREEN_CLR,
    ASSISTANT_ROLE:BLUE_CLR,
    FUNCTION_ROLE:MAGENTA_CLR
}

In [54]:
@retry(wait=wait_random_exponential(multiplier=1,max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages,functions=None,function_call=None,model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    }
    
    json_data = {"model":model,"messages":messages}
    if functions is not None:
        json_data.update({"functions":functions})
    if function_call is not None:
        json_data.update({"function_call":function_call})
        
    try:
        resp = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return resp
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [55]:
def pretty_print_conversation(messages):
    for message in messages:
        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == SYSTEM_ROLE:
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == USER_ROLE:
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == ASSISTANT_ROLE and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == ASSISTANT_ROLE and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == FUNCTION_ROLE:
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [ ]:
###functions 实际上定义的是规范化的返回值，根据返回值，通过function route，调用实际的函数

In [56]:
functions = [
    {
        "name":"get_current_weather",
        "description": "Get the current weather",  # 功能的描述
        "parameters":{
            "type":"object",
            "properties":{
                "location":{
                    "type":"string",
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format":{
                    "type":"string",
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                }
            },
            "required":["location","format"]
        }    
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        }
    }
]

In [66]:
messages = []

messages.append({"role":"system",
                 "content":"Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

In [67]:
chat_resp = chat_completion_request(messages,functions=functions)
print(chat_resp.json())
assistant_message = chat_resp.json()["choices"][0]["message"]
messages.append(assistant_message)
pretty_print_conversation(messages)

{'id': 'chatcmpl-9LQ0bgJeIP3QlKhW1BC359ouKYlL2', 'object': 'chat.completion', 'created': 1714891893, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Sure, could you please provide me with the location for which you would like to know the weather?'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 191, 'completion_tokens': 21, 'total_tokens': 212}, 'system_fingerprint': 'fp_a450710239'}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, could you please provide me with the location for which you would like to know the weather?



In [68]:
messages.append({
    "role":"user",
    "content":"I'm in Shanghai, China."
})

chat_resp = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_resp.json()["choices"][0]["message"]
messages.append(assistant_message)
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, could you please provide me with the location for which you would like to know the weather?

user: I'm in Shanghai, China.

assistant[function_call]: {'name': 'get_current_weather', 'arguments': '{"location":"Shanghai, China","format":"celsius"}'}



In [69]:
# 初始化一个空的messages列表
messages = []

# 向messages列表添加一条系统角色的消息，要求不做关于函数参数值的假设，如果用户的请求模糊，应该寻求澄清
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 向messages列表添加一条用户角色的消息，用户询问在未来x天内苏格兰格拉斯哥的天气情况
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "what is the weather going to be like in Shanghai, China over the next x days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure, I can help you with that. Please provide the number of days you would like the weather forecast for in Shanghai, China.



In [70]:
# 向messages列表添加一条用户角色的消息，用户指定接下来的天数为5天
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "5 days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure, I can help you with that. Please provide the number of days you would like the weather forecast for in Shanghai, China.

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":5}'}



In [71]:
# 在这个代码单元中，我们强制GPT 模型使用get_n_day_weather_forecast函数
messages = []  # 创建一个空的消息列表

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me a weather report for San Diego, USA."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages、functions以及特定的function_call作为参数
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for San Diego, USA.

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"San Diego, USA","format":"celsius","num_days":1}'}



In [72]:
# 在这个代码单元中，我们强制GPT 模型使用get_n_day_weather_forecast函数
messages = []  # 创建一个空的消息列表

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me a weather report for Beijing, China"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages、functions以及特定的function_call作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for Beijing, China

assistant[function_call]: {'name': 'get_current_weather', 'arguments': '{"location":"Beijing, China","format":"celsius"}'}



In [85]:
import sqlite3
conn = sqlite3.connect("data/chinook.db")

In [75]:
def get_table_names(conn):
    """返回一个包含所有表名的列表"""
    table_names = []  # 创建一个空的表名列表
    # 执行SQL查询，获取数据库中所有表的名字
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # 遍历查询结果，并将每个表名添加到列表中
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  # 返回表名列表


def get_column_names(conn, table_name):
    """返回一个给定表的所有列名的列表"""
    column_names = []  # 创建一个空的列名列表
    # 执行SQL查询，获取表的所有列的信息
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    # 遍历查询结果，并将每个列名添加到列表中
    for col in columns:
        column_names.append(col[1])
    return column_names  # 返回列名列表


def get_database_info(conn):
    """返回一个字典列表，每个字典包含一个表的名字和列信息"""
    table_dicts = []  # 创建一个空的字典列表
    # 遍历数据库中的所有表
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  # 获取当前表的所有列名
        # 将表名和列名信息作为一个字典添加到列表中
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  # 返回字典列表

In [76]:
db_info_dic = get_database_info(conn)
db_info_str = "\n".join(
    [
        f"Table: {t['table_name']}\nColumns:{', '.join(t['column_names'])}"
        for t in db_info_dic
    ]
)

In [77]:
print(db_info_str)

Table: albums
Columns:AlbumId, Title, ArtistId
Table: sqlite_sequence
Columns:name, seq
Table: artists
Columns:ArtistId, Name
Table: customers
Columns:CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: employees
Columns:EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: genres
Columns:GenreId, Name
Table: invoices
Columns:InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: invoice_items
Columns:InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: media_types
Columns:MediaTypeId, Name
Table: playlists
Columns:PlaylistId, Name
Table: playlist_track
Columns:PlaylistId, TrackId
Table: tracks
Columns:TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice
Table: sqlite_stat1
Columns:tbl, idx, stat


In [79]:
functions = [
    {
        "name":"ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters":{
            "type": "object",
            "properties":{
                "query":{
                    "type": "string",
                    "description":f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {db_info_str}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required":["query"]
        }
    }
]

In [95]:
def ask_database(conn,query):
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

In [97]:
def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results =  ask_database(conn,query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
        
    return results
    

In [100]:
# 创建一个空的消息列表
messages = []

# 向消息列表中添加一个系统角色的消息，内容是 "Answer user questions by generating SQL queries against the Chinook Music Database."
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})

# 向消息列表中添加一个用户角色的消息，内容是 "Hi, who are the top 5 artists by number of tracks?"
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, functions)

# 从聊天响应中获取助手的消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的消息添加到消息列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT artists.Name AS Artist, COUNT(tracks.TrackId) AS NumberOfTracks\\nFROM artists\\nJOIN albums ON artists.ArtistId = albums.ArtistId\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Artist\\nORDER BY NumberOfTracks DESC\\nLIMIT 5;"}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]



In [101]:
# 向消息列表中添加一个用户的问题，内容是 "What is the name of the album with the most tracks?"
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, functions)

# 从聊天响应中获取助手的消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的消息添加到消息列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT artists.Name AS Artist, COUNT(tracks.TrackId) AS NumberOfTracks\\nFROM artists\\nJOIN albums ON artists.ArtistId = albums.ArtistId\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Artist\\nORDER BY NumberOfTracks DESC\\nLIMIT 5;"}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT albums.Title AS Album, COUNT(tracks.TrackId) AS NumberOfTracks\\nFROM albums\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Album\\nORDER BY NumberOfTracks DESC\\nLIMIT 1;"}'}

function (ask_database): [('Greatest Hits', 57)]

